La función `PARSE_DOCUMENT` es una función específica de Cortex AI que permite extraer texto, datos y elementos de diseño de documentos. Podemos usar `PARSE_DOCUMENT` para extraer texto de diversos documentos y formularios e implementar:

- Pipelines RAG para impulsar búsquedas en Cortex Search
- Procesamiento LLM, como resumen o traducción de documentos, mediante funciones de Cortex AI

### Modos de uso de PARSE `_DOCUMENT`

La función `PARSE_DOCUMENT` ofrece dos modos para procesar documentos PDF:

- **El modo OCR** es la opción recomendada para la extracción de texto rápida y de alta calidad de documentos como manuales, contratos, páginas de detalles de productos, pólizas de seguro y reclamaciones, y documentos de SharePoint.
- **El modo LAYOUT** está optimizado para extraer texto y elementos de diseño como tablas. Se recomienda para mejorar el contexto de una base de conocimiento de documentos, optimizar los sistemas de recuperación de información y la inferencia del LLMs.

### Sintaxis

```sql
SNOWFLAKE.CORTEX.PARSE_DOCUMENT( '@<stage>', '<path>', [ { 'mode': '<mode>' }, ] )
```

### Ejemplo en modo OCR

Para este ejemplo crearemos un stage con las condiciones necesarias para utilizar `PARSE_DOCUMENT`. Luego cargaremos al stage algunos documentos PDF y los procesaremos.

In [ ]:
USE DATABASE curso_ia;
USE SCHEMA seccion_2;

CREATE OR REPLACE STAGE documentos
    DIRECTORY = ( ENABLE = true )
    ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' );

In [ ]:
USE DATABASE curso_ia;
USE SCHEMA seccion_2;

SELECT
  SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
    @documentos,
    'Que_es_la_IA.pdf',
    {'mode': 'OCR'}
  ) AS OCR;

Como se puede observar el resultado es un JSON que contiene la información extraida del PDF. Si deseamos extraer solo el contenido lo podemos hacer como sigue.

In [ ]:
USE DATABASE curso_ia;
USE SCHEMA seccion_2;

SELECT
  SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
    @documentos,
    'Que_es_la_IA.pdf',
    {'mode': 'OCR'}
  ):content AS OCR;

Como se observa el resultado ahora es solo el contenido del PDF.

### Ejemplo en modo LAYOUT

Para este ejemplo vamos a procesar un PDF que contiene varias tablas de ahí que utilicemos el modo LAYOUT para sacar el mayor beneficio al procesar el documento.

In [ ]:
USE DATABASE curso_ia;
USE SCHEMA seccion_2;

SELECT
  SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
    @documentos,
    'AI_Service_Reporte_Trimestral.pdf',
    {'mode': 'LAYOUT'}
  ):content AS LAYOUT;

### Combinar `PARSE_DOCUMENT` con `COMPLETE` para procesar documentos

En este ejemplo vamos a procesar los PDF que tenemos en el stage con la función `PARSE_DOCUMENT` y luego tomaremos el resultado para introducirlo a la función `COMPLETE` para obtener respuestas a preguntas sobre los documentos.

In [ ]:
CREATE OR REPLACE TABLE curso_ia.seccion_2.ia AS
    SELECT
        TO_VARCHAR(
            SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
                @documentos,
                'Que_es_la_IA.pdf',
                {'mode': 'OCR'}
            )
        ) AS texto;

In [ ]:
SELECT * FROM curso_ia.seccion_2.ia;

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'claude-3-5-sonnet',
    CONCAT(
        '¿Cuáles son las ventajas de la IA?',
        texto
    )
) FROM curso_ia.seccion_2.ia;

Tratemos ahora de extraer información del PDF con los estados financieros siguiendo la misma estrategia pero esta vez usando el modo LAYOUT y una tabla temporal.

In [ ]:
CREATE OR REPLACE TEMPORARY TABLE curso_ia.seccion_2.estados_financieros AS
    SELECT
        TO_VARCHAR(
            SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
                @documentos,
                'AI_Service_Reporte_Trimestral.pdf',
                {'mode': 'OCR'}
            )
        ) AS texto;

SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'claude-3-5-sonnet',
    CONCAT(
        '¿Cuál fue el promedio de ventas de los tres meses reportados? Solo responde con el resultado solicitado.',
        texto
    )
) FROM curso_ia.seccion_2.estados_financieros;

### Puntos importante sobre la función `PARSE_DOCUMENT`

- `PARSE_DOCUMENT` está optimizado para documentos tanto digitales como escaneados.
- Tamaño máximo de archivo 100 MB.
- Máximo de páginas por documento 300 páginas.
- Formatos de archivos permitidos: PDF, PPTX, DOCX, JPEG, JPG, PNG, TIFF, TIF.
- Se requiere un tamaño de fuente de 8 puntos o mayor para obtener mejores resultados.